In [ ]:
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize the images to a range of 0 to 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reshape the images to include the channel dimension
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))


In [ ]:
def model_builder(hp):
  inputs = tf.keras.Input(shape=(28, 28, 1))
  x = inputs
  for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
    filters = hp.Int('filters_' + str(i), 16, 128, step=16)
    for _ in range(2):
      x = tf.keras.layers.Convolution2D(
        filters, kernel_size=(3, 3), padding='same')(x)
      x = tf.keras.layers.BatchNormalization()(x)
      x = tf.keras.layers.ReLU()(x)
    if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
      x = tf.keras.layers.MaxPool2D()(x)
    else:
      x = tf.keras.layers.AvgPool2D()(x)
  x = tf.keras.layers.GlobalAvgPool2D()(x)
  x = tf.keras.layers.Dense(hp.Int('hidden_size', 30, 100, step=10, default=50),activation='relu')(x)
  x = tf.keras.layers.Dropout(hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
  outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

  model = tf.keras.Model(inputs, outputs)
  model.compile(
    optimizer=tf.keras.optimizers.Adam(
      hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  return model


def model_builder_v2(hp):
  inputs = tf.keras.Input(shape=(28, 28, 1))
  x = inputs
  for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
    filters = hp.Int('filters_' + str(i), 16, 128, step=16)
    dropout = hp.Float('dropout_' + str(i), 0, 0.8)
    for _ in range(2):
      x = tf.keras.layers.Convolution2D(filters, kernel_size=(3, 3), padding='same')(x)
      x = tf.keras.layers.SpatialDropout2D(dropout)(x)
      x = tf.keras.layers.BatchNormalization()(x)
      x = tf.keras.layers.ReLU()(x)
    if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
      x = tf.keras.layers.MaxPool2D((2, 2))(x)
    else:
      x = tf.keras.layers.AvgPool2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(10, (1, 1), padding="same",  activation='softmax')(x)
  outputs = tf.keras.layers.GlobalAvgPool2D()(x)
  model = tf.keras.Model(inputs, outputs)
  
  optimizers = {
    'adamW': tf.keras.optimizers.AdamW,
    'adam': tf.keras.optimizers.Adam,
    'nadam': tf.keras.optimizers.Nadam
  }
  optimizer_name = hp.Choice('optimizer', list(optimizers.keys()))
  lr = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
  model.compile(
    optimizer=optimizers[optimizer_name](learning_rate=lr),
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  return model


def only_optimizer(hp):
  model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), padding="same", activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.SpatialDropout2D(0.25), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), padding="same",  activation='relu'),
        tf.keras.layers.SpatialDropout2D(0.25), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), padding="same",  activation='relu'),
        tf.keras.layers.SpatialDropout2D(0.25), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(256, (3, 3), padding="same",  activation='relu'),
        tf.keras.layers.SpatialDropout2D(0.25), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(10, (1, 1), padding="same",  activation='softmax'), 
        tf.keras.layers.GlobalAvgPool2D(),
    ], name="global_avg")
  
  lr = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log', default=0.001)
  beta_1 = hp.Float('beta_1', 0.7, 0.99, sampling='log', default=0.9)
  beta_2 = hp.Float('beta_2', 0.9, 0.999, sampling='log', default=0.999)
  ema_momentum = hp.Float('ema_momentum', 0.9, 0.999, default=0.99)
  model.compile(
    optimizer=tf.keras.optimizers.Nadam(
      learning_rate=lr,
      beta_1=beta_1,
      beta_2=beta_2,
      use_ema=True,
      ema_momentum=ema_momentum),
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  return model
  

In [ ]:
# tuner = kt.Hyperband(model_builder_v2,
#                      objective='val_accuracy',
#                      max_epochs=10,
#                      factor=3,
#                      directory='my_dir',
#                      project_name='fashion_mnist_kt')
tuner = kt.BayesianOptimization(
    only_optimizer,
    objective='val_accuracy',
    max_trials=30,
    directory="optimizer_search",
    project_name='global_avg_model'
)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=0)

In [ ]:
tuner.search(train_images, train_labels, epochs=3, validation_split=0.2, callbacks=[stop_early])
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
best_hps.values

In [ ]:
import pandas as pd

# Get the search results
results = tuner.oracle.get_best_trials(num_trials=10)

# Prepare a DataFrame to store results
results_data = []
for trial in results:
    trial_data = trial.hyperparameters.values
    trial_data['score'] = trial.score
    results_data.append(trial_data)

df = pd.DataFrame(results_data)

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot for units vs. accuracy

plt.figure(figsize=(10, 6))
plt.scatter(df['learning_rate'], df['score'])
plt.xlabel('learning_rate')
plt.ylabel('Validation Accuracy')
plt.title('Units vs. Validation Accuracy')
plt.grid(True)

In [ ]:
best_hps.values

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_images, train_labels, epochs=30, validation_split=0.2)


In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()